**RLeXplore allows you to combine multiple intrinsic rewards to explore the potential  assembly advantages.**

**Load the libraries**

In [ ]:
import gymnasium as gym
import numpy as np
import torch as th

from rllte.env.utils import Gymnasium2Torch
from rllte.xplore.reward import Fabric, RE3, ICM
from rllte.agent import PPO

**Create a fake Atari environment with image observations**

In [ ]:
class FakeAtari(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Discrete(7)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(4, 84, 84))
        self.count = 0

    def reset(self):
        self.count = 0
        return self.observation_space.sample(), {}

    def step(self, action):
        self.count += 1
        if self.count > 100 and np.random.rand() < 0.1:
            term = trunc = True
        else:
            term = trunc = False
        return self.observation_space.sample(), 0, term, trunc, {}

**Use the `Fabric` class to create a mixed intrinsic reward**

In [ ]:
# set the parameters
device = 'cuda' if th.cuda.is_available() else 'cpu'
n_steps = 128
n_envs = 8
# create the vectorized environments
envs = gym.vector.AsyncVectorEnv([FakeAtari for _ in range(n_envs)])
# wrap the environments to convert the observations to torch tensors
envs = Gymnasium2Torch(envs, device)
# create two intrinsic reward functions
irs1 = ICM(envs, device)
irs2 = RE3(envs, device)
# create the mixed intrinsic reward function
irs = Fabric([irs1, irs2])

In [ ]:
# start the training
device = 'cuda' if th.cuda.is_available() else 'cpu'
# create the PPO agent
agent = PPO(envs, device=device)
# set the intrinsic reward module
agent.set(reward=irs)
# train the agent
agent.train(10000)